In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install talib-binary

In [ ]:
from BinanceClient import get_data

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import os
import sys
import pandas as pd
import datetime as dt
import time
import numpy as np
import talib
from talib import MA_Type
import matplotlib.pyplot as plt
import gc 
from IPython.core.display import clear_output
from tqdm import tqdm

In [ ]:
kline_df = pd.read_csv("drive/MyDrive/KLINE_INTERVAL_5MINUTE.csv")

In [ ]:
#instrument = "BTC"
#start_date = dt.datetime(2020,1,1,0,0,0)
#interval = "5m"
#kline_df = get_data(
#        currency=instrument+"USDT",
#        start_date=start_date,
#        interval=interval)

In [ ]:
period = 20

In [ ]:
kline_df["upper_BB"], kline_df["lower_BB"],kline_df["middle_BB"] = talib.BBANDS(
            kline_df.close,
            timeperiod=period,
            nbdevup=3,
            nbdevdn=2,
            matype=0
        )

In [ ]:
kline_df["ma"] = talib.EMA(kline_df["close"], timeperiod=period)

kline_df['ADX0'] = talib.ADX(
        kline_df.high,
        kline_df.low,
        kline_df.close,
        timeperiod=period #20
    )

kline_df['ATR0'] = talib.ATR(
    kline_df.high,
    kline_df.low,
    kline_df.close,
    timeperiod=period
)

#kline_df['ARO0'] = talib.AROONOSC(
#high=kline_df.high,
#low=kline_df.low,
#timeperiod=period
#)

kline_df['WLR0'] = talib.WILLR(
    kline_df.high,
    kline_df.low,
    kline_df.close,
    timeperiod=period
)

In [ ]:
kline_df = kline_df.dropna()
kline_df.head()

In [ ]:
def normalize(arr):
  mini=np.min(arr)
  arr -= mini
  maxi = np.max(arr)
  arr /= maxi
  return arr

In [ ]:
X = []
Y = []
Y_old = []
for i in tqdm(range(20, kline_df.shape[0]-6, 2)):
  x = []
  Y_old.append(kline_df["close"].iloc[i + 6])
  Y.append(normalize(Y_old)[-1])
  for column in kline_df.columns[1:]:
    x.append(normalize(np.array(kline_df[column][i-20:i])))
  X.append(x)

In [ ]:
X = np.ascontiguousarray(X, dtype=np.float16)
Y = np.ascontiguousarray(Y, dtype=np.float16)

In [ ]:
Y = Y[1:]
X = X[1:]

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))

In [ ]:
inputs = keras.Input(shape=(X.shape[1], X.shape[2], X.shape[3]),dtype=tf.float16)
x1 = layers.Conv2D(32, 2, activation="relu")(inputs)
x1 = layers.Conv2D(32, 2, activation="relu")(x1)
x1 = layers.Conv2D(32, 2, activation="relu")(x1)
x1 = layers.MaxPooling2D(2)(x1)
x1 = layers.Conv2D(32, 2, activation="relu")(x1)
x1 = layers.MaxPooling2D(2)(x1)


x2 = layers.Conv2D(32, 2, activation="relu")(inputs)
x2 = layers.Conv2D(32, 2, activation="relu")(x2)
x2 = layers.Conv2D(32, 2, activation="relu")(x2)
x2 = layers.MaxPooling2D(2)(x2)
x2 = layers.Conv2D(32, 2, activation="relu")(x2)
x2 = layers.MaxPooling2D(2)(x2)


x3 = layers.Conv2D(32, 2, activation="relu")(inputs)
x3 = layers.Conv2D(32, 2, activation="relu")(x3)
x3 = layers.Conv2D(32, 2, activation="relu")(x3)
x3 = layers.MaxPooling2D(2)(x3)
x3 = layers.Conv2D(32, 2, activation="relu")(x3)
x3 = layers.MaxPooling2D(2)(x3)

x = tf.keras.layers.Concatenate(axis=1)([x1, x2, x3])
x = layers.Flatten()(x)
x = dense = layers.Dense(128, activation="relu")(x)
x = dense = layers.Dense(64, activation="relu")(x)
x = dense = layers.Dense(32, activation="relu")(x)
x = dense = layers.Dense(16, activation="relu")(x)
x = dense = layers.Dense(8, activation="relu")(x)
x = dense = layers.Dense(4, activation="relu")(x)
x = dense = layers.Dense(2, activation="relu")(x)


outputs = dense = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="algotrading")

In [ ]:
model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.RMSprop())
model.summary()

In [ ]:
model.fit(X, Y, batch_size=64, epochs=10, validation_split=0.2)